In [1]:
import os

import pandas as pd
import numpy as np

In [2]:
raw_loc_base = '../data/raw'

In [3]:
master_name = 'Input information and X10 - X18.xlsx'
master_loc = os.path.join(raw_loc_base, master_name)
master = pd.read_excel(master_loc, sheet='inputs', header=0)

In [4]:
master.head()

,Input,Key,X10: Category Method,X11: Temperature (K),X12: [Salt*Valency],X13: Category Salt type,X14: [Buffer] (mM),X15: pH,X16: CI number,X17: CI,X18: CP,Output: logK
0,1,17,A,298,0,0,0,6.00,0,NaN,CP_3,4.39
1,2,17,B,298,0,0,50,4.74,0,NaN,NaN,4.39
2,3,19,A,298,0,0,0,6.00,1,CI_2,CP_3,4.63
3,4,20,A,298,0,0,0,6.00,2,CI_1,CP_3,4.80
4,5,21,A,298,0,0,0,6.00,2,CI_2,CP_3,4.90


In [5]:
def read_X1(file_name, sheet_name='X1', col_names=['atom_number', 'AMU', 'x', 'y', 'z']):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None, names=col_names)
    
def read_X2(file_name, sheet_name='X2', col_names=['occupied', 'virtual']):
    # TODO: reformat
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    return df

def read_X3(file_name, sheet_name='X3'):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None)

def read_X4(file_name, sheet_name='X4'):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None)

def read_X5(file_name, sheet_name='X5', col_names=['electric_potential', 'x', 'y', 'z']):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None, names=col_names)

def read_X6(file_name, sheet_name='X6', col_names=['XX', 'YY', 'ZZ', 'XY', 'XZ', 'YZ']):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None, names=col_names)

def read_X7(file_name, sheet_name='X7'):
    return pd.read_excel(file_name, sheet_name='X7', header=None)

def read_X8(file_name, sheet_name='X8', col_name='SCF'):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    flat = df.values.flatten()
    return pd.Series(flat[~np.isnan(flat)]).rename(col_name)
    
def read_X9(file_name, sheet_name='X9', col_name='MO'):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    flat = df.values.flatten()
    return pd.Series(flat[~np.isnan(flat)]).rename(col_name)

In [6]:
x = read_X9(os.path.join(raw_loc_base, '19.xlsx'))
x

0        -8.416098e-07
1         4.763023e-06
2        -8.087775e-05
3         3.542323e-05
4        -1.318132e-05
5         1.370000e-04
6         4.301706e-04
7        -1.777651e-04
8         7.243307e-05
9        -1.005905e-02
10        3.550072e-03
11       -3.081912e-03
12        5.097859e-04
13       -2.101435e-03
14        4.218406e-03
15        1.410838e-03
16        8.896886e-04
17        2.259254e-05
18       -2.553621e-05
19        8.795824e-06
20       -9.531010e-05
21        3.978579e-05
22       -5.568704e-06
23        2.589919e-05
24       -3.435383e-05
25       -1.724554e-06
26       -4.242745e-06
27       -2.861062e-04
28        2.216206e-04
29        1.132570e-06
              ...     
105920    2.804653e-04
105921    2.047755e-04
105922   -1.509222e-04
105923    3.554436e-03
105924    1.662724e-03
105925    1.299279e-02
105926   -1.300804e-04
105927   -7.935511e-04
105928   -4.080188e-04
105929    2.140280e-04
105930    2.060922e-04
105931    3.480167e-03
105932    6